In [3]:
import os

from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken

from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console


In [5]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

finish_reason='stop' content='The capital of France is Paris.' usage=RequestUsage(prompt_tokens=14, completion_tokens=7) cached=False logprobs=None thought=None


In [6]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[],
    system_message="You are a travel agent that plans great vacations",
)

In [7]:
async def assistant_run() -> None:
    response = await agent.on_messages(
        [TextMessage(content="Plan me a great sunny vacation", source="user")],
        cancellation_token=CancellationToken(),
    )
    print(response.inner_messages)
    print(response.chat_message)


# Use asyncio.run(assistant_run()) when running in a script.
await assistant_run()

[]
source='assistant' models_usage=RequestUsage(prompt_tokens=26, completion_tokens=646) content="Absolutely! Here’s a sunny vacation plan that combines relaxation, adventure, and cultural experiences. Let’s set you up for a week in beautiful Costa Rica!\n\n### Destination: Costa Rica\n\n#### Duration: 7 Days\n\n#### Ideal Travel Time: December to April (dry season)\n\n---\n\n### Day 1: Arrival in San José\n- **Flight:** Arrive at Juan Santamaría International Airport (SJO).\n- **Accommodation:** Check into a trendy hotel in the Escazú or downtown San José.\n- **Evening:** Enjoy dinner at a local restaurant featuring Costa Rican cuisine. Try “casado” (rice and beans with protein).\n\n---\n\n### Day 2: Arenal Volcano\n- **Morning:** Travel to La Fortuna (about 3 hours from San José).\n- **Accommodation:** Stay in a resort with views of Arenal Volcano.\n- **Afternoon:** Visit the La Fortuna Waterfall for a refreshing swim.\n- **Evening:** Relax in the hot springs at Tabacon or Baldi.\n\n